<a href="https://colab.research.google.com/github/dgsob/MT7051-VT25/blob/main/Group%20project/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MuJoCo Set Up

In [1]:
# Install mujoco
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess

# Check if GPU is available
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config for Nvidia EGL driver
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was successful
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl
Checking that the installation succeeded:
Installation successful.


# Gym and Other Libraries

In [2]:
!pip install gymnasium
!pip install gymnasium-robotics

import gymnasium as gym
import gymnasium_robotics as robotics

# Some things for rendering
!apt-get install -y xvfb
!pip install imageio ffmpeg pyvirtualdisplay
import imageio
from IPython.display import display
from PIL import Image

# Probably needed for something
import numpy as np

# PyTorch for some RL or shit
import torch

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.12).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


# Define env and visualization

In [3]:
PUSH = "FetchPush-v3"
SLIDE = "FetchSlide-v3"
REACH = "FetchReach-v3"
PICK_AND_PLACE = "FetchPickAndPlace-v3"

In [4]:
def create_env(env_name):
    """Creates and returns a Gymnasium environment.

    Args:
        env_name (string): The name of the environment to create.
        Valid options are:
            - "FetchPush-v3"
            - "FetchSlide-v3"
            - "FetchReach-v3"
            - "FetchPickAndPlace-v3"

    Returns:
        gym.Env: The created Gymnasium environment.

    Prints:
        Observation space: The observation space of the environment.
        Action space: The action space of the environment.
    """
    env = gym.make(env_name, render_mode="rgb_array")
    print("Observation space:", env.observation_space)
    print("Action space:", env.action_space)
    return env

In [5]:
def save_gif(frames, filename="fetch.gif", fps=10):
    """Save frames as a GIF."""
    imageio.mimsave(filename, frames, fps=fps)
    print(f"Saved animation to {filename}")

# Run a random agent to test the env

In [6]:
def run_random_actions(env, steps):
    """Run a random agent in the environment for a given number of steps."""
    frames = []
    observation, info = env.reset()

    for _ in range(steps):
        action = env.action_space.sample()  # Sample a random action
        obs, reward, terminated, truncated, info = env.step(action)

        # Capture frame
        frames.append(env.render())

        # Stop if episode ends
        if terminated or truncated:
            break

    env.close()
    return frames

In [7]:
# Run everything
env = create_env(PUSH)
frames = run_random_actions(env, steps=1000)
save_gif(frames)

Observation space: Dict('achieved_goal': Box(-inf, inf, (3,), float64), 'desired_goal': Box(-inf, inf, (3,), float64), 'observation': Box(-inf, inf, (25,), float64))
Action space: Box(-1.0, 1.0, (4,), float32)
Saved animation to fetch.gif
